In [1]:
pip install pandas

In [2]:
pip install numpy

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
dados = pd.read_csv('/content/sample_data/tested.csv')

In [5]:
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
dados = dados.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis = 1)

In [7]:
dados.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,0,3,male,34.5,0,0,7.8292
1,893,1,3,female,47.0,1,0,7.0000
2,894,0,2,male,62.0,0,0,9.6875
3,895,0,3,male,27.0,0,0,8.6625
4,896,1,3,female,22.0,1,1,12.2875


Setando PassengerId como Chave e variável resposta Survived como target

In [8]:
dados = dados.set_index(['PassengerId'])
dados = dados.rename(columns= {'Survived': 'target'}, inplace = False)

In [9]:
dados.head()

,target,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
892,0,3,male,34.5,0,0,7.8292
893,1,3,female,47.0,1,0,7.0000
894,0,2,male,62.0,0,0,9.6875
895,0,3,male,27.0,0,0,8.6625
896,1,3,female,22.0,1,1,12.2875


Descritiva

In [10]:
dados.describe()

,target,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


Transformação de dados

In [11]:
dados['Sex_F'] = np.where(dados['Sex'] == 'female', 1, 0)
dados['Pclass_1'] = np.where(dados['Pclass'] == 1, 1, 0)
dados['Pclass_2'] = np.where(dados['Pclass'] == 2, 1, 0)
dados['Pclass_3'] = np.where(dados['Pclass'] == 3, 1, 0)

In [12]:
dados = dados.drop(['Pclass', 'Sex'], axis = 1)

In [13]:
dados.head()

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,
892,0,34.5,0,0,7.8292,0,0,0,1
893,1,47.0,1,0,7.0000,1,0,0,1
894,0,62.0,0,0,9.6875,0,0,1,0
895,0,27.0,0,0,8.6625,0,0,0,1
896,1,22.0,1,1,12.2875,1,0,0,1


Onde não tem valores atribui 0

In [14]:
dados.isnull().sum()

target       0
Age         86
SibSp        0
Parch        0
Fare         1
Sex_F        0
Pclass_1     0
Pclass_2     0
Pclass_3     0
dtype: int64

In [15]:
dados.fillna(0, inplace = True)

In [16]:
dados.isnull().sum()

target      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex_F       0
Pclass_1    0
Pclass_2    0
Pclass_3    0
dtype: int64

Amostragem

70% da base para treinar o modelo e 30% para testar o modelo

In [23]:
x_train, x_test, y_train, y_test = train_test_split(dados.drop(['target'], axis = 1),
                                                    dados['target'],
                                                    test_size = 0.3,
                                                    random_state = 1234)
[{'treino': x_train.shape}, {'teste': x_test.shape}]

[{'treino': (292, 10)}, {'teste': (126, 10)}]

Modelo - Random Forest

Em português, Random Forest significa floresta aleatória. Este nome explica muito bem o funcionamento do algoritmo, que irá criar muitas árvores de decisão, de maneira aleatória, formando o que podemos enxergar como uma floresta, onde cada árvore será utilizada na escolha do resultado final... (https://didatica.tech/o-que-e-e-como-funciona-o-algoritmo-randomforest/)

Faça 1000 árvores, com o critério 'gini' e com profundidade 5

Primeiro mapeia o modelo, depois realiza os cálculos com .fit

In [24]:
rndforest = RandomForestClassifier(n_estimators = 1000,
                                   criterion = 'gini',
                                   max_depth = 5)
rndforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
probabilidade = rndforest.predict_proba(dados.drop('target', axis = 1))[:,1]
classificacao = rndforest.predict(dados.drop('target', axis = 1))

In [26]:
dados['probabilidade'] = probabilidade
dados['classificacao'] = classificacao

In [27]:
dados

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3,probabilidade,classificacao
PassengerId,,,,,,,,,,,
892,0,34.5,0,0,7.8292,0,0,0,1,0.000152,0
893,1,47.0,1,0,7.0000,1,0,0,1,0.997909,1
894,0,62.0,0,0,9.6875,0,0,1,0,0.002000,0
895,0,27.0,0,0,8.6625,0,0,0,1,0.000152,0
896,1,22.0,1,1,12.2875,1,0,0,1,0.999000,1
...,...,...,...,...,...,...,...,...,...,...,...
1305,0,0.0,0,0,8.0500,0,0,0,1,0.000280,0
1306,1,39.0,0,0,108.9000,1,1,0,0,0.999094,1
1307,0,38.5,0,0,7.2500,0,0,0,1,0.001061,0
